# Calculate special functions

see Book Elementarteilchenphysik, Berger, Herten
Section 2.2 - 2.4

Topics:
- d-function, Tab. 2.1, eq. 2.53, 2.54
- D-function, eq. 2.56
- special harmonic functions $Y_l^m$ (from book and from sympy)

- calculate resonance production as discussed in the book
    - $\pi p \to \Delta \to \pi p$
    - $\pi\pi \to \rho \to \pi\pi$
    - $gg\to H\to \gamma\gamma$
    - $gg\to H\to Z\gamma$

- print all d- and D-function for J
- check some equations in the book 

---

## Functions

In [2]:
import sympy as sy

In [3]:
theta, phi = sy.symbols('theta phi ',real=True)

In [4]:
# define some sympy abbreviations
ct = sy.cos(theta)
st = sy.sin(theta)
ct2 = sy.cos(theta/2)
st2 = sy.sin(theta/2)
r12 = sy.Rational(1,2)
r32 = sy.Rational(3,2)
r14 = sy.Rational(1,4)

# dfunc[(J,MP,M)] from the book Tab. 2.1
dfunc = {
    # J=0
    (0,0,0): 1,
    # J=1/2
    (r12,r12,r12):ct2, 
    (r12,r12,-r12):-st2,
    # J=1
    (1,1,1):(1+ct)/2, 
    (1,1,0):-st/sy.sqrt(2), 
    (1,1,-1):(1-ct)/2, 
    (1,0,0):ct,
    # J=3/2
    (r32,r32,r32): (1+ct)*ct2/2, 
    (r32,r32,r12): -sy.sqrt(3)*(1+ct)*st2/2,
    (r32,r32,-r12): sy.sqrt(3)*(1-ct)*ct2/2,
    (r32,r32,-r32): -(1-ct)*st2/2,
    (r32,r12,r12) : (3*ct-1)*ct2/2,
    (r32,r12,-r12) : -(3*ct+1)*st2/2,
    # J=2
    (2,2,2): r14*(1+ct)**2,
    (2,2,1): -r12*(1+ct)*st,
    (2,2,0): r14*sy.sqrt(6)*st**2,
    (2,2,-1): -r12*(1-ct)*st,
    (2,2,-2): r14*(1-ct)**2,
    (2,1,1) : r12*(1+ct)*(2*ct-1),
    (2,1,0) : -sy.sqrt(sy.Rational(3,2))*st*ct,
    (2,1,-1) : r12*(1-ct)*(2*ct+1),
    (2,0,0) : r32*ct**2-r12
}

In [5]:
def dfunction(J,MP,M):
    """
    Calculate d-function 

    Parameters
    J (integer or sy.Rational): spin
    MP (integer or sy.Rational): M^\prime
    MP (integer or sy.Rational): M

    Return 
    d-function (sy.Rational)


    # use eq. 2.53, 2.54 to calculate missing combinations
    """
    if (J,MP,M) in dfunc.keys():
        return dfunc[(J,MP,M)]
    elif (J,-M,-MP) in dfunc.keys():
        return dfunc[(J,-M,-MP)]
    elif (J,M,MP) in dfunc.keys():
        return (-1)**(M-MP)*dfunc[(J,M,MP)]
    elif (J,-MP,-M) in dfunc.keys():
        return (-1)**(M-MP)*dfunc[(J,-MP,-M)]
    else:
        print('Error for: ',J,MP,M)
        return None

        
    


In [6]:
def Dfunction(J,MP,M):
    """
    Return D-function, see eq. 2.56
    """
    return sy.exp(-sy.I * phi * (MP-M) ) * dfunction(J,MP,M)

In [7]:
def printd(J):
    # print all d-functions and D-functions for angular momentum J
    mul = int(2*J+1.1)
    step = 1
    for ip in range(mul):
        MP = -J + ip*step
        for i in range(mul):
            M = -J + i*step
            d = sy.simplify(dfunction(J,MP,M))
            D = sy.simplify(Dfunction(J,MP,M))
            print('J,MP,M: ',J,MP,M,' d: ', d,' D: ',Dfunction(J,MP,M))



In [8]:
def Ylm(l,m):  # Y_l^m
    """Spherical harmonics
    
    Functions given in the Book, Table 2.2 and unnumbered equation after 2.57"
    """
    dY = {(0,0): 1/sy.sqrt(4*sy.pi),
          (1,0): sy.sqrt(sy.Rational(3,4)/sy.pi) * sy.cos(theta),
          (1,1): -sy.sqrt(sy.Rational(3,8)/sy.pi) * sy.sin(theta) * sy.exp(sy.I*phi),
          (2,0): sy.sqrt(sy.Rational(5,4)/sy.pi) * (sy.Rational(3,2)*sy.cos(theta)**2-r12),
          (2,1): -sy.sqrt(sy.Rational(15,8)/sy.pi) * sy.sin(theta) * sy.cos(theta) * sy.exp(sy.I*phi),
          (2,2): sy.sqrt(sy.Rational(15,32)/sy.pi) * sy.sin(theta)**2 * sy.exp(sy.I*2*phi)
    }
    t=(l,m)
    if m >= 0:
          return dY[t]
    else:
          tt = (l,-m)
          return (-1)**abs(m) * dY[tt].conjugate()

In [9]:
def Ynm(n,m):
    """Spherical Harmonics from sympy
    
    see: https://docs.sympy.org/latest/modules/functions/special.html#spherical-harmonics
    """
    return sy.Ynm(n,m,theta,phi).expand(func=True)

---

# Examples

### 1) Delta resonance

In [ ]:
# eq. 2.109, p. 132
dsdO = dfunction(r32,r12,r12)**2 + dfunction(r32,r12,-r12)**2 
dsdO = sy.simplify(dsdO)
dsdO.subs(sy.cos(2*theta),2*sy.cos(theta)**2-1)

3*cos(theta)**2/4 + 1/4

### 2) $\rho\to \pi \pi$ decay

In [25]:
# rho -> pi pi , page 134
dfunction(1,0,0)**2

cos(theta)**2

### 3) $gg \to H\to \gamma\gamma$ for J=2

$d^J_{\lambda\mu}$ with $\lambda=\lambda_1 - \lambda_2, \mu=\lambda_3-\lambda_4$ (eq. 2.89, 2.106)

In [ ]:
# Higgs decay into gamma  gamma, p. 137
# gluons have helicity lambda_i = -1,1
# photons have lambda_i = -1,1
# therefore 
# lamda = lambda_1 - lambda_2 = (-2,0,2) ; 
# mu = lambda_3 - lambda_4 = (-2,0,2)

display('(2,0,0): ',(dfunction(2,0,0)**2))
display('(2,0,2): ',(dfunction(2,0,2)**2))
r = r12*(dfunction(2,2,2)**2+dfunction(2,2,-2)**2).expand()
r = r.subs(sy.sin(theta)**4,(1-sy.cos(theta)**2)**2).expand()
display('(2,2,2)+(2,2-2): ',r)

'(2,0,0): '

(3*cos(theta)**2/2 - 1/2)**2

'(2,0,2): '

3*sin(theta)**4/8

'(2,2,2)+(2,2-2): '

cos(theta)**4/16 + 3*cos(theta)**2/8 + 1/16

### 4) $gg \to H\to Z\gamma$ for J=2

Exercise 2.23

$d^J_{\lambda\mu}$ with $\lambda=\lambda_1 - \lambda_2, ~\mu=\lambda_3-\lambda_4$ (eq. 2.89, 2.106)

In [26]:
# exercise 2.23, p. 137
# H > Z gamma for H with J=2
# gluons have lambda_i = -1,1
# photons have lambda_i = -1,1
# Z has lambda_i = -1,0,1
# therefore lamda = (0, ± 2) ; mu= (0, ± 1, ± 2) 
r= sy.simplify(r12*(dfunction(2,2,1)**2+dfunction(2,2,-1)**2))
display('(2,2,1): ',r)
display('(2,0,1): ',(dfunction(2,0,1)**2))

'(2,2,1): '

(cos(theta)**2 + 1)*sin(theta)**2/4

'(2,0,1): '

3*sin(theta)**2*cos(theta)**2/2

# Print d and D functions

In [ ]:
# Spin 0
printd(0)

J,MP,M:  0 0 0  d:  1  D:  1


In [ ]:
# Spin 1/2
printd(r12)

J,MP,M:  1/2 -1/2 -1/2  d:  cos(theta/2)  D:  cos(theta/2)
J,MP,M:  1/2 -1/2 1/2  d:  sin(theta/2)  D:  exp(I*phi)*sin(theta/2)
J,MP,M:  1/2 1/2 -1/2  d:  -sin(theta/2)  D:  -exp(-I*phi)*sin(theta/2)
J,MP,M:  1/2 1/2 1/2  d:  cos(theta/2)  D:  cos(theta/2)


In [ ]:
# Spin 1
printd(1)

J,MP,M:  1 -1 -1  d:  cos(theta)/2 + 1/2  D:  cos(theta)/2 + 1/2
J,MP,M:  1 -1 0  d:  sqrt(2)*sin(theta)/2  D:  sqrt(2)*exp(I*phi)*sin(theta)/2
J,MP,M:  1 -1 1  d:  1/2 - cos(theta)/2  D:  (1/2 - cos(theta)/2)*exp(2*I*phi)
J,MP,M:  1 0 -1  d:  -sqrt(2)*sin(theta)/2  D:  -sqrt(2)*exp(-I*phi)*sin(theta)/2
J,MP,M:  1 0 0  d:  cos(theta)  D:  cos(theta)
J,MP,M:  1 0 1  d:  sqrt(2)*sin(theta)/2  D:  sqrt(2)*exp(I*phi)*sin(theta)/2
J,MP,M:  1 1 -1  d:  1/2 - cos(theta)/2  D:  (1/2 - cos(theta)/2)*exp(-2*I*phi)
J,MP,M:  1 1 0  d:  -sqrt(2)*sin(theta)/2  D:  -sqrt(2)*exp(-I*phi)*sin(theta)/2
J,MP,M:  1 1 1  d:  cos(theta)/2 + 1/2  D:  cos(theta)/2 + 1/2


In [ ]:
# Spin 3/2
printd(r32)

J,MP,M:  3/2 -3/2 -3/2  d:  (cos(theta) + 1)*cos(theta/2)/2  D:  (cos(theta) + 1)*cos(theta/2)/2
J,MP,M:  3/2 -3/2 -1/2  d:  sqrt(3)*(cos(theta) + 1)*sin(theta/2)/2  D:  sqrt(3)*(cos(theta) + 1)*exp(I*phi)*sin(theta/2)/2
J,MP,M:  3/2 -3/2 1/2  d:  sqrt(3)*(1 - cos(theta))*cos(theta/2)/2  D:  sqrt(3)*(1 - cos(theta))*exp(2*I*phi)*cos(theta/2)/2
J,MP,M:  3/2 -3/2 3/2  d:  (1 - cos(theta))*sin(theta/2)/2  D:  -(cos(theta) - 1)*exp(3*I*phi)*sin(theta/2)/2
J,MP,M:  3/2 -1/2 -3/2  d:  -sqrt(3)*(cos(theta) + 1)*sin(theta/2)/2  D:  -sqrt(3)*(cos(theta) + 1)*exp(-I*phi)*sin(theta/2)/2
J,MP,M:  3/2 -1/2 -1/2  d:  (3*cos(theta) - 1)*cos(theta/2)/2  D:  (3*cos(theta) - 1)*cos(theta/2)/2
J,MP,M:  3/2 -1/2 1/2  d:  (3*cos(theta) + 1)*sin(theta/2)/2  D:  -(-3*cos(theta) - 1)*exp(I*phi)*sin(theta/2)/2
J,MP,M:  3/2 -1/2 3/2  d:  sqrt(3)*(1 - cos(theta))*cos(theta/2)/2  D:  sqrt(3)*(1 - cos(theta))*exp(2*I*phi)*cos(theta/2)/2
J,MP,M:  3/2 1/2 -3/2  d:  sqrt(3)*(1 - cos(theta))*cos(theta/2)/2  D:  sqrt(3

In [ ]:
# Spin 2
printd(2)

J,MP,M:  2 -2 -2  d:  (cos(theta) + 1)**2/4  D:  (cos(theta) + 1)**2/4
J,MP,M:  2 -2 -1  d:  (cos(theta) + 1)*sin(theta)/2  D:  -(-cos(theta)/2 - 1/2)*exp(I*phi)*sin(theta)
J,MP,M:  2 -2 0  d:  sqrt(6)*sin(theta)**2/4  D:  sqrt(6)*exp(2*I*phi)*sin(theta)**2/4
J,MP,M:  2 -2 1  d:  (1 - cos(theta))*sin(theta)/2  D:  -(cos(theta)/2 - 1/2)*exp(3*I*phi)*sin(theta)
J,MP,M:  2 -2 2  d:  (cos(theta) - 1)**2/4  D:  (1 - cos(theta))**2*exp(4*I*phi)/4
J,MP,M:  2 -1 -2  d:  -(cos(theta) + 1)*sin(theta)/2  D:  (-cos(theta)/2 - 1/2)*exp(-I*phi)*sin(theta)
J,MP,M:  2 -1 -1  d:  cos(theta)/2 + cos(2*theta)/2  D:  (cos(theta)/2 + 1/2)*(2*cos(theta) - 1)
J,MP,M:  2 -1 0  d:  sqrt(6)*sin(2*theta)/4  D:  sqrt(6)*exp(I*phi)*sin(theta)*cos(theta)/2
J,MP,M:  2 -1 1  d:  cos(theta)/2 - cos(2*theta)/2  D:  (1/2 - cos(theta)/2)*(2*cos(theta) + 1)*exp(2*I*phi)
J,MP,M:  2 -1 2  d:  (1 - cos(theta))*sin(theta)/2  D:  -(cos(theta)/2 - 1/2)*exp(3*I*phi)*sin(theta)
J,MP,M:  2 0 -2  d:  sqrt(6)*sin(theta)**2/4  D:  sq

# Checks

### 1) Check: eq. 2.57


In [ ]:
def DY(J,MP):
    # calculate D^j_(j3,0) in eq. 2.57 from spherical harmonics
    DY1 = sy.sqrt(4*sy.pi/(2*J+1)) * Ylm(J,MP).conjugate()
    return  DY1

In [ ]:
# lJMPM = [(j, j3', j3=0), ... ]
lJMPM =[(0,0,0),(1,1,0),(1,0,0),(1,-1,0),(2,2,0),(2,1,0),(2,0,0),(2,-1,0),(2,-2,0)]
# compare Dfunction and DY
print("(j,j3', j3) | Equal | Dfunction() | DY()")
for t in lJMPM:
    s = f'{t} | {Dfunction(*t)==DY(*t[:2])} | {Dfunction(*t)} | {DY(*t[:2])}'
    print(s)

(j,j3', j3) | Equal | Dfunction() | DY()
(0, 0, 0) | True | 1 | 1
(1, 1, 0) | True | -sqrt(2)*exp(-I*phi)*sin(theta)/2 | -sqrt(2)*exp(-I*phi)*sin(theta)/2
(1, 0, 0) | True | cos(theta) | cos(theta)
(1, -1, 0) | True | sqrt(2)*exp(I*phi)*sin(theta)/2 | sqrt(2)*exp(I*phi)*sin(theta)/2
(2, 2, 0) | True | sqrt(6)*exp(-2*I*phi)*sin(theta)**2/4 | sqrt(6)*exp(-2*I*phi)*sin(theta)**2/4
(2, 1, 0) | True | -sqrt(6)*exp(-I*phi)*sin(theta)*cos(theta)/2 | -sqrt(6)*exp(-I*phi)*sin(theta)*cos(theta)/2
(2, 0, 0) | True | 3*cos(theta)**2/2 - 1/2 | 3*cos(theta)**2/2 - 1/2
(2, -1, 0) | True | sqrt(6)*exp(I*phi)*sin(theta)*cos(theta)/2 | sqrt(6)*exp(I*phi)*sin(theta)*cos(theta)/2
(2, -2, 0) | True | sqrt(6)*exp(2*I*phi)*sin(theta)**2/4 | sqrt(6)*exp(2*I*phi)*sin(theta)**2/4


### 2) Compare spherical harmonic function $Y_n^m$  from sympy with $Y_l^m$ in the book

In [27]:
#  get spherical harmonics from sympy
# 
sy.Ynm(2,1,theta,phi).expand(func=True)

-sqrt(30)*exp(I*phi)*sin(theta)*cos(theta)/(4*sqrt(pi))

In [28]:
Ylm(2,1)

-sqrt(30)*exp(I*phi)*sin(theta)*cos(theta)/(4*sqrt(pi))

In [ ]:
# compare both function of spherical harmonics
# lY = [(l, m), ... ] or [(n, m), ... ]
print('(l,m) | Equal | Ynm | Ylm')
lY =[(0,0),(1,1),(1,0),(1,-1),(2,2),(2,1),(2,0),(2,-1),(2,-2)]
for t in lY:
    ynm = sy.simplify(sy.Ynm(*t,theta,phi).expand(func=True))
    ylm = sy.simplify(Ylm(*t))
    s = f"{t} | {ynm == ylm} | {ynm} | {ylm}"
    print(s)

(l,m) | Equal | Ynm | Ylm
(0, 0) | True | 1/(2*sqrt(pi)) | 1/(2*sqrt(pi))
(1, 1) | True | -sqrt(6)*exp(I*phi)*sin(theta)/(4*sqrt(pi)) | -sqrt(6)*exp(I*phi)*sin(theta)/(4*sqrt(pi))
(1, 0) | True | sqrt(3)*cos(theta)/(2*sqrt(pi)) | sqrt(3)*cos(theta)/(2*sqrt(pi))
(1, -1) | True | sqrt(6)*exp(-I*phi)*sin(theta)/(4*sqrt(pi)) | sqrt(6)*exp(-I*phi)*sin(theta)/(4*sqrt(pi))
(2, 2) | True | sqrt(30)*exp(2*I*phi)*sin(theta)**2/(8*sqrt(pi)) | sqrt(30)*exp(2*I*phi)*sin(theta)**2/(8*sqrt(pi))
(2, 1) | True | -sqrt(30)*exp(I*phi)*sin(2*theta)/(8*sqrt(pi)) | -sqrt(30)*exp(I*phi)*sin(2*theta)/(8*sqrt(pi))
(2, 0) | True | sqrt(5)*(3*cos(theta)**2 - 1)/(4*sqrt(pi)) | sqrt(5)*(3*cos(theta)**2 - 1)/(4*sqrt(pi))
(2, -1) | True | sqrt(30)*exp(-I*phi)*sin(2*theta)/(8*sqrt(pi)) | sqrt(30)*exp(-I*phi)*sin(2*theta)/(8*sqrt(pi))
(2, -2) | True | sqrt(30)*exp(-2*I*phi)*sin(theta)**2/(8*sqrt(pi)) | sqrt(30)*exp(-2*I*phi)*sin(theta)**2/(8*sqrt(pi))
